## Week 12 Exercise 12.2 Author: Rex Gayas Course & Section: DSC360-T301 Data Mining: Text Analytics an (2243-1) Date: 02 MAR 2024

##### Loading the Dataset

In [62]:
import pandas as pd

# Define the file path
file_path = "D:\\ALPHA\\Dynamic Folder\\Bellevue\\Winter 2023\\Data Mining\\Week 5\\twitter_sample.csv"

# Load the dataset into a Pandas DataFrame
tweets_df = pd.read_csv(file_path)

# Display the first few rows of the dataframe to verify
print(tweets_df.head())


                Tweet Id                                          Tweet URL  \
0  "1167429261210218497"  https://twitter.com/animalhealthEU/status/1167...   
1  "1167375334670557185"  https://twitter.com/PennyBrohnUK/status/116737...   
2  "1167237977615097861"  https://twitter.com/lordbyronaf/status/1167237...   
3  "1167236897078480898"  https://twitter.com/CountessDavis/status/11672...   
4  "1167228378191204353"  https://twitter.com/Local12/status/11672283781...   

  Tweet Posted Time (UTC)                                      Tweet Content  \
0    30 Aug 2019 13:30:00  Pets change our lives &amp; become a part of o...   
1    30 Aug 2019 09:55:43  Another spot of our #morethanmedicine bus in #...   
2    30 Aug 2019 00:49:54  What a great team ⁦@HealthSourceOH⁩ ⁦@Local12⁩...   
3    30 Aug 2019 00:45:37  What a great team ⁦@HealthSourceOH⁩ ⁦@Local12⁩...   
4    30 Aug 2019 00:11:46  What a great team ⁦@HealthSourceOH⁩ ⁦@Local12⁩...   

  Tweet Type                Client  Retweets

##### Preparing the spaCy Pipeline

In [63]:
import spacy

# Load the English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")


##### Define Matcher and Patterns for Social Causes &  Implement the on_match Callback

In [82]:
import spacy
from spacy.matcher import Matcher, PhraseMatcher

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize the Matcher with the spaCy vocabulary
matcher = Matcher(nlp.vocab)

# Define an on_match callback function
def on_match_callback(matcher, doc, id, matches):
    match_id, start, end = matches[0]
    span = doc[start:end]
    print(f"Match found: {span.text}, at Position: {start}-{end}")

# Define patterns for social causes
social_cause_patterns = [
    [{"LOWER": "support"}, {"IS_PUNCT": True, "OP": "?"}, {"LOWER": "the"}, {"LOWER": "cause"}],
    [{"LOWER": "raise"}, {"LOWER": "awareness"}],
    [{"LOWER": "fight"}, {"LOWER": "against"}],
    [{"LOWER": "advocate"}, {"LOWER": "for"}],
    [{"LOWER": "social"}, {"LOWER": "justice"}],
    [{"LOWER": "human"}, {"LOWER": "rights"}],
    [{"LOWER": "help"}, {"LOWER": "the"}, {"LOWER": "poor"}],
    [{"LOWER": "poverty"}, {"LOWER": "relief"}],
    [{"LOWER": "equality"}],
    [{"LOWER": "environmental"}, {"LOWER": "protection"}],
    [{"LOWER": "climate"}, {"LOWER": "change"}],
    [{"LOWER": "political"}, {"LOWER": "reform"}],
    [{"LOWER": "donate"}, {"LOWER": "to"}],
    [{"LOWER": "charity"}],
    [{"LOWER": "volunteer"}, {"LOWER": "for"}],
    [{"LOWER": "fundraiser"}],
    [{"LOWER": "make"}, {"LOWER": "a"}, {"LOWER": "difference"}]
]
# Add patterns to the matcher with an on_match callback
matcher.add("SOCIAL_CAUSE", social_cause_patterns, on_match=on_match_callback)


##### Define the PhraseMatcher

In [83]:
# Initialize PhraseMatcher with the same vocabulary as the nlp object
phrase_matcher = PhraseMatcher(nlp.vocab)

# Define some phrases related to social causes
social_causes_phrases = [nlp(text) for text in ["climate change", "human rights", "social justice"]]

# Add the phrases to the PhraseMatcher
phrase_matcher.add("SOCIAL_CAUSE_PHRASES", social_causes_phrases)


##### Define Regular Expression Pattern

In [84]:
# Add a regular expression pattern to the matcher
matcher.add("HASHTAG", [[{"TEXT": {"REGEX": "#\w+"}}]])


##### Process Tweets and Extract Matches

In [85]:
# Function to process tweets and extract matches
def extract_social_causes(tweet_content):
    doc = nlp(tweet_content)
    matches = matcher(doc) + phrase_matcher(doc)
    matches = sorted(matches, key=lambda x: x[1])  # Sort matches by start index
    for match_id, start, end in matches:
        on_match_callback(matcher, doc, match_id, [(match_id, start, end)])
    return ', '.join([doc[start:end].text for match_id, start, end in matches])

# Apply the function to the 'Tweet Content' column
tweets_df['Matched Text'] = tweets_df['Tweet Content'].apply(extract_social_causes)

# Filter out rows with no matches
tweets_with_matches = tweets_df[tweets_df['Matched Text'] != '']

# Create a new DataFrame with the relevant columns
matches_df = tweets_with_matches[['Name', 'Matched Text']].copy()

# Add the 'Identifier' column with the constant value 'SOCIAL_CAUSE'
matches_df['Identifier'] = 'SOCIAL_CAUSE'

# Display the DataFrame
print(matches_df.head())





Match found: raise awareness, at Position: 24-26
Match found: raise awareness, at Position: 24-26
Match found: family!#morethanmedicine, at Position: 40-41
Match found: family!#morethanmedicine, at Position: 40-41
Match found: environmental protection, at Position: 38-40
Match found: environmental protection, at Position: 38-40
Match found: environmental protection, at Position: 38-40
Match found: environmental protection, at Position: 38-40
                   Name              Matched Text    Identifier
125                NOAH           raise awareness  SOCIAL_CAUSE
219         Andy Little  family!#morethanmedicine  SOCIAL_CAUSE
220         Drew Kalnow  family!#morethanmedicine  SOCIAL_CAUSE
359      Ibrahima KOUMA  environmental protection  SOCIAL_CAUSE
380  AnimalhealthEurope  environmental protection  SOCIAL_CAUSE
